In [3]:
# Imports
import os
from django.apps import apps as django_apps

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [4]:
from flourish_child.models import Appointment

child_idx = 'B142-040990678-0-10'
lower_visit = 2004
visit = 2007

while visit >= lower_visit:
    try:
        old_visit = ChildVisit.objects.get(subject_identifier=child_idx, visit_code=visit)
    except ChildVisit.DoesNotExist:
        print(visit)
        break
    else:
        appt = old_visit.appointment
        new_visit_code = visit + 1
        new_appt = Appointment.objects.filter(
            subject_identifier=child_idx, visit_code=new_visit_code)
        new_appt.update(
            appt_datetime=appt.appt_datetime,
            appt_type=appt.appt_type,
            appt_status=appt.appt_status,
            appt_reason=appt.appt_reason)
        old_visit.appointment = new_appt[0]
        old_visit.save()
        visit -= 1

In [5]:
Appointment.objects.filter(subject_identifier=child_idx, visit_code='2004').update(appt_status='in_progress')

1